To run this notebook you first need to run the notebook `evaluation/compute_metrics.ipynb` or obtain the data generated for such notebook in the corresponding folders
---

In [1]:
from json import dump, load
from pprint import pprint
from statistics import median, mean, stdev
import networkx as nx

## Load gene graphs

f = open(f'../gene_graphs/vertices_inv.json', 'r')
vertices_inv = load(f)
f.close()
vertices_inv = {int(k): tuple(v) for k,v in vertices_inv.items()}

components = list()
for i in range(19641): ## Number of gene_graphs
    components.append(dict())
    components[i]['graph'] = nx.read_edgelist(f'../gene_graphs/graphs/component_{i+1}.edgelist', delimiter=':', create_using=nx.DiGraph, nodetype=int)
    components[i]['len'] = len(components[i]['graph'])
    
    f = open(f'../gene_graphs/sources/component_{i+1}.json', 'r')
    components[i]['sources'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/targets/component_{i+1}.json', 'r')
    components[i]['targets'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/vertex_constrains/component_{i+1}.json', 'r')
    components[i]['vertex_constrains'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/transcript_paths/component_{i+1}.json', 'r')
    components[i]['transcript_paths'] = load(f)
    f.close()
    

In [2]:
## Load experiments from json files
for i, component in enumerate(components):
    if component['len'] > 2:
        file = open(f'../safe_paths_json/component_{i+1}.json', "r")
        dd = load(file)
        component.update(dd)
        file.close()

In [3]:
def interval_length(interval):
    return interval[1]-interval[0]+1

## It computes the base length of a transcript
def base_length(contig, vertices_inv):
    length = 0
    for v in contig:
        length += interval_length(vertices_inv[v])
    return length

In [4]:
def choose_60_30_10_limits(components, vertices_inv):
    lengths = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            for transcript in component['transcript_paths']:
                lengths.append(base_length(transcript, vertices_inv))
    lengths = sorted(lengths)
    return lengths[int(0.6*len(lengths))], lengths[int(0.9*len(lengths))], lengths[-1]

limit_60, limit_30, limit_10 = choose_60_30_10_limits(components, vertices_inv)
limit_60, limit_30, limit_10

(1941, 3617, 22168)

In [5]:
def choose_60_30_10_limits_size(components, vertices_inv):
    sizes = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            sizes.append(component['len'])
    sizes = sorted(sizes)
    return sizes[int(0.6*len(sizes))], sizes[int(0.9*len(sizes))], sizes[-1]

limit_size_60, limit_size_30, limit_size_10 = choose_60_30_10_limits_size(components, vertices_inv)
limit_size_60, limit_size_30, limit_size_10

(20, 56, 279)

In [6]:
def compute_abs_improvement_table(over_width, limit_60, limit_30, components, vertices_inv):
    unitigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)

            safe_paths = component['experiments'][str(l)]['safe_paths']
            uni = component['unitigs']
            
            abs_vertex = component['experiments'][str(l)]['impr_vertex']
            abs_base = component['experiments'][str(l)]['impr_base']
            
            abs_vertex = list(map(lambda r:  r[1] - len(uni[r[0]]), enumerate(abs_vertex)))
            abs_base = list(map(lambda r:  r[1] - base_length(uni[r[0]], vertices_inv), enumerate(abs_base)))
            
                
            for j in range(len(abs_base)):
                d = dict()
                d['component'] = i
                d['component_length'] = component['len']
                d['abs_improvements_base'] = abs_base[j]
                d['abs_improvements_vertex'] = abs_vertex[j]
                
                unitigs.append(d)

    small = list(filter(lambda c: c['component_length'] <= limit_60, unitigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , unitigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, unitigs))
    
    small_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , small))
    medium_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , medium))
    large_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , large))
    total_abs_improvements_base = list(map(lambda c: c['abs_improvements_base'] , unitigs))
    
    small_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , small))
    medium_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , medium))
    large_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , large))
    total_abs_improvements_vertex = list(map(lambda c: c['abs_improvements_vertex'] , unitigs))
    
    
    
    
    return { 
        'small':
        {
            'abs_improvements_vertex': 
            {
                'median': median(small_abs_improvements_vertex),
                'mean': mean(small_abs_improvements_vertex),
                'stdev': stdev(small_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(small_abs_improvements_base),
                'mean': mean(small_abs_improvements_base),
                'stdev': stdev(small_abs_improvements_base)
            },
            'number_of_unitigs': len(small)            
        },
        'medium':
        {
            'abs_improvements_vertex': 
            {
                'median': median(medium_abs_improvements_vertex),
                'mean': mean(medium_abs_improvements_vertex),
                'stdev': stdev(medium_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(medium_abs_improvements_base),
                'mean': mean(medium_abs_improvements_base),
                'stdev': stdev(medium_abs_improvements_base)
            },
            'number_of_unitigs': len(medium)
            
        },
        'large':
        {
            'abs_improvements_vertex': 
            {
                'median': median(large_abs_improvements_vertex),
                'mean': mean(large_abs_improvements_vertex),
                'stdev': stdev(large_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(large_abs_improvements_base),
                'mean': mean(large_abs_improvements_base),
                'stdev': stdev(large_abs_improvements_base)
            },
            'number_of_unitigs': len(large)
            
        },
        'total':
        {
            'abs_improvements_vertex': 
            {
                'median': median(total_abs_improvements_vertex),
                'mean': mean(total_abs_improvements_vertex),
                'stdev': stdev(total_abs_improvements_vertex)
            },
            'abs_improvements_base':
            {
                'median': median(total_abs_improvements_base),
                'mean': mean(total_abs_improvements_base),
                'stdev': stdev(total_abs_improvements_base)
            },
            'number_of_unitigs': len(unitigs)
            
        }
    }
    
pprint(compute_abs_improvement_table(0, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'abs_improvements_base': {'mean': 719.1604133141357,
                                     'median': 171.0,
                                     'stdev': 1521.2809117145976},
           'abs_improvements_vertex': {'mean': 9.577496400440417,
                                       'median': 3.0,
                                       'stdev': 18.611030611088154},
           'number_of_unitigs': 23614},
 'medium': {'abs_improvements_base': {'mean': 932.0984700568265,
                                      'median': 294,
                                      'stdev': 1481.2516966618873},
            'abs_improvements_vertex': {'mean': 8.09310797027539,
                                        'median': 3,
                                        'stdev': 11.24971491033276},
            'number_of_unitigs': 34315},
 'small': {'abs_improvements_base': {'mean': 788.8116951254412,
                                     'median': 405.0,
                                     'stdev': 1090.49

In [7]:
def compute_rel_improvement_table(over_width, limit_60, limit_30, components, vertices_inv):
    unitigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)

            uni = component['unitigs']
            safe_paths = component['experiments'][str(l)]['safe_paths']
            
            rel_vertex = component['experiments'][str(l)]['impr_vertex']
            rel_base = component['experiments'][str(l)]['impr_base']
            
            rel_vertex = list(map(lambda r:  r[1]/len(uni[r[0]]), enumerate(rel_vertex)))
            rel_base = list(map(lambda r:  r[1]/base_length(uni[r[0]], vertices_inv), enumerate(rel_base)))
            
            
                
            for j in range(len(rel_base)):
                d = dict()
                d['component'] = i
                d['component_length'] = component['len']
                d['rel_improvements_base'] = rel_base[j]
                d['rel_improvements_vertex'] = rel_vertex[j]
                
                unitigs.append(d)

    small = list(filter(lambda c: c['component_length'] <= limit_60, unitigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , unitigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, unitigs))
    
    small_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , small))
    medium_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , medium))
    large_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , large))
    total_rel_improvements_base = list(map(lambda c: c['rel_improvements_base'] , unitigs))
    
    small_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , small))
    medium_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , medium))
    large_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , large))
    total_rel_improvements_vertex = list(map(lambda c: c['rel_improvements_vertex'] , unitigs))
    
    
    
    
    return { 
        'small':
        {
            'rel_improvements_vertex': 
            {
                'median': median(small_rel_improvements_vertex),
                'mean': mean(small_rel_improvements_vertex),
                'stdev': stdev(small_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(small_rel_improvements_base),
                'mean': mean(small_rel_improvements_base),
                'stdev': stdev(small_rel_improvements_base)
            },
            'number_of_unitigs': len(small)            
        },
        'medium':
        {
            'rel_improvements_vertex': 
            {
                'median': median(medium_rel_improvements_vertex),
                'mean': mean(medium_rel_improvements_vertex),
                'stdev': stdev(medium_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(medium_rel_improvements_base),
                'mean': mean(medium_rel_improvements_base),
                'stdev': stdev(medium_rel_improvements_base)
            },
            'number_of_unitigs': len(medium)
            
        },
        'large':
        {
            'rel_improvements_vertex': 
            {
                'median': median(large_rel_improvements_vertex),
                'mean': mean(large_rel_improvements_vertex),
                'stdev': stdev(large_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(large_rel_improvements_base),
                'mean': mean(large_rel_improvements_base),
                'stdev': stdev(large_rel_improvements_base)
            },
            'number_of_unitigs': len(large)
            
        },
        'total':
        {
            'rel_improvements_vertex': 
            {
                'median': median(total_rel_improvements_vertex),
                'mean': mean(total_rel_improvements_vertex),
                'stdev': stdev(total_rel_improvements_vertex)
            },
            'rel_improvements_base':
            {
                'median': median(total_rel_improvements_base),
                'mean': mean(total_rel_improvements_base),
                'stdev': stdev(total_rel_improvements_base)
            },
            'number_of_unitigs': len(unitigs)
            
        }
    }
    
pprint(compute_rel_improvement_table(-1, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'number_of_unitigs': 23614,
           'rel_improvements_base': {'mean': 2.848581712347052,
                                     'median': 1.3929266259032795,
                                     'stdev': 8.509968108942164},
           'rel_improvements_vertex': {'mean': 1.622909721073752,
                                       'median': 1.3333333333333333,
                                       'stdev': 0.7978246619049792}},
 'medium': {'number_of_unitigs': 34315,
            'rel_improvements_base': {'mean': 2.639517935757701,
                                      'median': 1.1464435146443515,
                                      'stdev': 10.929682265629102},
            'rel_improvements_vertex': {'mean': 1.4602363101871334,
                                        'median': 1.3333333333333333,
                                        'stdev': 0.6350774325631543}},
 'small': {'number_of_unitigs': 20966,
           'rel_improvements_base': {'mean': 1.925980778419804,
      

In [8]:
def compute_contigs_table(over_width, limit_60, limit_30, components, vertices_inv):
    contigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]
                
                for safe_path in safe_paths_info['safe_paths']:
                    d = dict()
                    d['component'] = i
                    d['component_length'] = component['len']
                    d['vertices'] = len(safe_path)
                    d['length'] = base_length(safe_path, vertices_inv)
                    
                    contigs.append(d)

                
                    
    
    
   
    small = list(filter(lambda c: c['component_length'] <= limit_60, contigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , contigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, contigs))
    
    small_base_lengths = list(map(lambda c: c['length'] , small))
    medium_base_lengths = list(map(lambda c: c['length'] , medium))
    large_base_lengths = list(map(lambda c: c['length'] , large))
    total_base_lengths = list(map(lambda c: c['length'] , contigs))
    
    small_vertex_lengths = list(map(lambda c: c['vertices'] , small))
    medium_vertex_lengths = list(map(lambda c: c['vertices'] , medium))
    large_vertex_lengths = list(map(lambda c: c['vertices'] , large))
    total_vertex_lengths = list(map(lambda c: c['vertices'] , contigs))
    
    
    
    
    return { 
        'small':
        {
            'vertex_lengths': 
            {
                'median': median(small_vertex_lengths),
                'mean': mean(small_vertex_lengths),
                'stdev': stdev(small_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(small_base_lengths),
                'mean': mean(small_base_lengths),
                'stdev': stdev(small_base_lengths)
            },
            'number_of_contigs': len(small)            
        },
        'medium':
        {
            'vertex_lengths': 
            {
                'median': median(medium_vertex_lengths),
                'mean': mean(medium_vertex_lengths),
                'stdev': stdev(medium_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(medium_base_lengths),
                'mean': mean(medium_base_lengths),
                'stdev': stdev(medium_base_lengths)
            },
            'number_of_contigs': len(medium)
            
        },
        'large':
        {
            'vertex_lengths': 
            {
                'median': median(large_vertex_lengths),
                'mean': mean(large_vertex_lengths),
                'stdev': stdev(large_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(large_base_lengths),
                'mean': mean(large_base_lengths),
                'stdev': stdev(large_base_lengths)
            },
            'number_of_contigs': len(large)
            
        },
        'total':
        {
            'vertex_lengths': 
            {
                'median': median(total_vertex_lengths),
                'mean': mean(total_vertex_lengths),
                'stdev': stdev(total_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(total_base_lengths),
                'mean': mean(total_base_lengths),
                'stdev': stdev(total_base_lengths)
            },
            'number_of_contigs': len(contigs)
            
        }
    }
    
pprint(compute_contigs_table(-1, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'base_lengths': {'mean': 436.4558600008216,
                            'median': 309,
                            'stdev': 447.04617673255626},
           'number_of_contigs': 24343,
           'vertex_lengths': {'mean': 5.720658916320914,
                              'median': 5,
                              'stdev': 3.682998267698851}},
 'medium': {'base_lengths': {'mean': 567.7217631593068,
                             'median': 417.0,
                             'stdev': 534.0104072524672},
            'number_of_contigs': 37274,
            'vertex_lengths': {'mean': 4.867736223641144,
                               'median': 4.0,
                               'stdev': 2.357499016890991}},
 'small': {'base_lengths': {'mean': 774.8026468381199,
                            'median': 601.0,
                            'stdev': 655.8812098329743},
           'number_of_contigs': 29318,
           'vertex_lengths': {'mean': 3.559826727607613,
                           

In [9]:
def compute_contigs_table_unitigs(limit_60, limit_30, components, vertices_inv):
    contigs = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            unitigs = component['unitigs']

            for unitig in unitigs:
                d = dict()
                d['component'] = i
                d['component_length'] = component['len']
                d['vertices'] = len(unitig)
                d['length'] = base_length(unitig, vertices_inv)

                contigs.append(d)
                
    
    
    small = list(filter(lambda c: c['component_length'] <= limit_60, contigs))
    medium = list(filter(lambda c: c['component_length'] > limit_60 and c['component_length'] <= limit_30 , contigs))
    large = list(filter(lambda c: c['component_length'] > limit_30, contigs))
    
    small_base_lengths = list(map(lambda c: c['length'] , small))
    medium_base_lengths = list(map(lambda c: c['length'] , medium))
    large_base_lengths = list(map(lambda c: c['length'] , large))
    total_base_lengths = list(map(lambda c: c['length'] , contigs))
    
    small_vertex_lengths = list(map(lambda c: c['vertices'] , small))
    medium_vertex_lengths = list(map(lambda c: c['vertices'] , medium))
    large_vertex_lengths = list(map(lambda c: c['vertices'] , large))
    total_vertex_lengths = list(map(lambda c: c['vertices'] , contigs))
    
    
    
    
    return { 
        'small':
        {
            'vertex_lengths': 
            {
                'median': median(small_vertex_lengths),
                'mean': mean(small_vertex_lengths),
                'stdev': stdev(small_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(small_base_lengths),
                'mean': mean(small_base_lengths),
                'stdev': stdev(small_base_lengths)
            },
            'number_of_contigs': len(small)            
        },
        'medium':
        {
            'vertex_lengths': 
            {
                'median': median(medium_vertex_lengths),
                'mean': mean(medium_vertex_lengths),
                'stdev': stdev(medium_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(medium_base_lengths),
                'mean': mean(medium_base_lengths),
                'stdev': stdev(medium_base_lengths)
            },
            'number_of_contigs': len(medium)
            
        },
        'large':
        {
            'vertex_lengths': 
            {
                'median': median(large_vertex_lengths),
                'mean': mean(large_vertex_lengths),
                'stdev': stdev(large_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(large_base_lengths),
                'mean': mean(large_base_lengths),
                'stdev': stdev(large_base_lengths)
            },
            'number_of_contigs': len(large)
            
        },
        'total':
        {
            'vertex_lengths': 
            {
                'median': median(total_vertex_lengths),
                'mean': mean(total_vertex_lengths),
                'stdev': stdev(total_vertex_lengths)
            },
            'base_lengths':
            {
                'median': median(total_base_lengths),
                'mean': mean(total_base_lengths),
                'stdev': stdev(total_base_lengths)
            },
            'number_of_contigs': len(contigs)
            
        }
    }

pprint(compute_contigs_table_unitigs(limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'base_lengths': {'mean': 330.63682561192513,
                            'median': 215.0,
                            'stdev': 404.1071037259436},
           'number_of_contigs': 23614,
           'vertex_lengths': {'mean': 4.420343863809604,
                              'median': 3.0,
                              'stdev': 3.371163388398297}},
 'medium': {'base_lengths': {'mean': 448.1594929331196,
                             'median': 312,
                             'stdev': 480.16626703307566},
            'number_of_contigs': 34315,
            'vertex_lengths': {'mean': 4.0131429404050705,
                               'median': 3,
                               'stdev': 1.966312076649829}},
 'small': {'base_lengths': {'mean': 696.1803872937136,
                            'median': 521.0,
                            'stdev': 629.8592824260448},
           'number_of_contigs': 20966,
           'vertex_lengths': {'mean': 3.4202995325765526,
                        

In [10]:
def compute_fixed_l_table(over_width, limit_60, limit_30, components, vertices_inv):
    transcripts = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]

                for j, transcript in enumerate(component['transcript_paths']):
                    d = dict()
                    d['component'] = i
                    d['length'] = base_length(transcript, vertices_inv)
                    d['transcript'] = transcript
                    
                    d['e_size_density_vertex'] = safe_paths_info['e_sizes_vertex'][j]/len(transcript)
                    d['e_size_density_bases'] = safe_paths_info['e_size_bases'][j]/d['length']
                    d['max_prop_cov_bases'] = safe_paths_info['max_cov_bases'][j]/d['length']
                    d['max_prop_cov_vertex'] = safe_paths_info['max_cov_vertex'][j]/len(transcript)
                    
                    transcripts.append(d)
    
    
    
    small = list(filter(lambda t: t['length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['length'] > limit_60 and t['length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['length'] > limit_30, transcripts))
    
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))
    
    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            }
        }
    }
    
pprint(compute_fixed_l_table(0, limit_60, limit_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.31476876311319757,
                                    'median': 0.24297949134571306,
                                    'stdev': 0.23331538376491084},
           'e_size_density_vertex': {'mean': 0.2372338144930291,
                                     'median': 0.1712962962962963,
                                     'stdev': 0.20385849276163728},
           'max_prop_cov_bases': {'mean': 0.452046092145866,
                                  'median': 0.4065325261421473,
                                  'stdev': 0.23394008701797325},
           'max_prop_cov_vertex': {'mean': 0.3479379624184463,
                                   'median': 0.2909090909090909,
                                   'stdev': 0.20063122484879942}},
 'medium': {'e_size_density_bases': {'mean': 0.3413025293449205,
                                     'median': 0.26537749128443533,
                                     'stdev': 0.2338210361354344},
            'e_s

In [11]:
def compute_fixed_l_table_unitigs(limit_60, limit_30, components, vertices_inv):
    transcripts = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            unitigs = component['unitigs']

            for j, transcript in enumerate(component['transcript_paths']):
                d = dict()
                d['component'] = i
                d['length'] = base_length(transcript, vertices_inv)
                d['transcript'] = transcript
                
                
                d['e_size_density_vertex'] =  component['e_sizes_vertex'][j]/len(transcript)
                d['e_size_density_bases'] =  component['e_size_bases'][j]/d['length']
                d['max_prop_cov_bases'] =  component['max_cov_bases'][j]/d['length']
                d['max_prop_cov_vertex'] =  component['max_cov_vertex'][j]/len(transcript)

                transcripts.append(d)

    
    
    small = list(filter(lambda t: t['length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['length'] > limit_60 and t['length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['length'] > limit_30, transcripts))
    
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))

    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            }
        }
    }

pprint(compute_fixed_l_table_unitigs(limit_60, limit_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.20831605008517648,
                                    'median': 0.16277244927362938,
                                    'stdev': 0.1572681159947799},
           'e_size_density_vertex': {'mean': 0.13022390356167357,
                                     'median': 0.1111111111111111,
                                     'stdev': 0.08880200898799885},
           'max_prop_cov_bases': {'mean': 0.3457419077553817,
                                  'median': 0.3081544284416954,
                                  'stdev': 0.18371228766695671},
           'max_prop_cov_vertex': {'mean': 0.2443698887203478,
                                   'median': 0.22641509433962265,
                                   'stdev': 0.11208902594662568}},
 'medium': {'e_size_density_bases': {'mean': 0.23978768898062428,
                                     'median': 0.18464356442993646,
                                     'stdev': 0.17715792872635627},
            

In [12]:
def compute_fixed_rd_table(over_width, limit_60, limit_30, components, vertices_inv):
    genes = list()
    transcripts = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
                
                
            if component['experiments'].get(l, None) is None:
                l = str(2*width)
                        
            if component['experiments'].get(l, None) is not None:
                safe_paths_info = component['experiments'][l]

                for j, transcript in enumerate(component['transcript_paths']):
                    d = dict()
                    d['component'] = i
                    d['component_length'] = component['len']
                    d['length'] = base_length(transcript, vertices_inv)
                    d['transcript'] = transcript
                    
                    
                    d['e_size_density_vertex'] = safe_paths_info['e_sizes_vertex'][j]/len(transcript)
                    d['e_size_density_bases'] = safe_paths_info['e_size_bases'][j]/d['length']
                    d['max_prop_cov_bases'] = safe_paths_info['max_cov_bases'][j]/d['length']
                    d['max_prop_cov_vertex'] = safe_paths_info['max_cov_vertex'][j]/len(transcript)
                    
                    transcripts.append(d)
                 
                d = dict()
                
                tp_bases = 0
                tp_vertex = 0
                for safe_path in safe_paths_info['true_positives']:
                    tp_bases += base_length(safe_path, vertices_inv)
                    tp_vertex += len(safe_path)
                    
                p_bases = tp_bases
                p_vertex = tp_vertex
                for safe_path in safe_paths_info['false_positives']:
                    p_bases += base_length(safe_path, vertices_inv)
                    p_vertex += len(safe_path)
                
                d['precision_bases'] = 1.0*tp_bases/p_bases
                d['precision_vertex'] = 1.0*tp_vertex/p_vertex
                d['component_length'] = components[i]['len']
                genes.append(d)
    
    small = list(filter(lambda t: t['component_length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['component_length'] > limit_30, transcripts))
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))
    
    
    s = list(filter(lambda t: t['component_length'] <= limit_60, genes))
    m = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , genes))
    l = list(filter(lambda t: t['component_length'] > limit_30, genes))
    
    s_precisions_bases = list(map(lambda t: t['precision_bases'] , s))
    m_precisions_bases = list(map(lambda t: t['precision_bases'] , m))
    l_precisions_bases = list(map(lambda t: t['precision_bases'] , l))
    
    s_precisions_vertex = list(map(lambda t: t['precision_vertex'] , s))
    m_precisions_vertex = list(map(lambda t: t['precision_vertex'] , m))
    l_precisions_vertex = list(map(lambda t: t['precision_vertex'] , l))
    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(s_precisions_bases),
                'mean': mean(s_precisions_bases),
                'stdev': stdev(s_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(s_precisions_vertex),
                'mean': mean(s_precisions_vertex),
                'stdev': stdev(s_precisions_vertex)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(m_precisions_bases),
                'mean': mean(m_precisions_bases),
                'stdev': stdev(m_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(m_precisions_vertex),
                'mean': mean(m_precisions_vertex),
                'stdev': stdev(m_precisions_vertex)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(l_precisions_bases),
                'mean': mean(l_precisions_bases),
                'stdev': stdev(l_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(l_precisions_vertex),
                'mean': mean(l_precisions_vertex),
                'stdev': stdev(l_precisions_vertex)
            }
        }
    }
    
pprint(compute_fixed_rd_table(0, limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.2513150234496567,
                                    'median': 0.19761129901342084,
                                    'stdev': 0.1734499395888049},
           'e_size_density_vertex': {'mean': 0.21547436328360273,
                                     'median': 0.17421140353001593,
                                     'stdev': 0.14964659138235403},
           'max_prop_cov_bases': {'mean': 0.38729566137468546,
                                  'median': 0.34218203977279493,
                                  'stdev': 0.18755029641255325},
           'max_prop_cov_vertex': {'mean': 0.3386141936372565,
                                   'median': 0.30303030303030304,
                                   'stdev': 0.16159291235603568},
           'precision_bases': {'mean': 0.7847857229303314,
                               'median': 0.8903138054988827,
                               'stdev': 0.28356221782283764},
           'precision_vertex': 

In [13]:
def compute_fixed_rd_table_unitigs(limit_60, limit_30, components, vertices_inv):
    transcripts = list()
    genes = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            unitigs = component['unitigs']

            for j, transcript in enumerate(component['transcript_paths']):
                d = dict()
                d['component'] = i
                d['component_length'] = components[i]['len']
                d['length'] = base_length(transcript, vertices_inv)
                d['transcript'] = transcript
                
                
                d['e_size_density_vertex'] =  component['e_sizes_vertex'][j]/len(transcript)
                d['e_size_density_bases'] =  component['e_size_bases'][j]/d['length']
                d['max_prop_cov_bases'] =  component['max_cov_bases'][j]/d['length']
                d['max_prop_cov_vertex'] =  component['max_cov_vertex'][j]/len(transcript)

                transcripts.append(d)
                
            d = dict()
                
            tp_bases = 0
            tp_vertex = 0
            for safe_path in component['true_positives']:
                tp_bases += base_length(safe_path, vertices_inv)
                tp_vertex += len(safe_path)

            p_bases = tp_bases
            p_vertex = tp_vertex
            for safe_path in component['false_positives']:
                p_bases += base_length(safe_path, vertices_inv)
                p_vertex += len(safe_path)

            d['precision_bases'] = 1.0 if p_bases == 0 else 1.0*tp_bases/p_bases
            d['precision_vertex'] = 1.0 if p_vertex == 0 else 1.0*tp_vertex/p_vertex
            d['component_length'] = component['len']
            genes.append(d)

    
    
    small = list(filter(lambda t: t['component_length'] <= limit_60, transcripts))
    medium = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , transcripts))
    large = list(filter(lambda t: t['component_length'] > limit_30, transcripts))
    
    small_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , small))
    medium_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , medium))
    large_e_size_densities_vertex = list(map(lambda t: t['e_size_density_vertex'] , large))
    
    small_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , small))
    medium_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , medium))
    large_e_size_densities_bases = list(map(lambda t: t['e_size_density_bases'] , large))
    
    
    small_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , small))
    medium_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , medium))
    large_max_prop_cov_vertex = list(map(lambda t: t['max_prop_cov_vertex'] , large))
    
    small_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , small))
    medium_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , medium))
    large_max_prop_cov_bases = list(map(lambda t: t['max_prop_cov_bases'] , large))
    
    
    s = list(filter(lambda t: t['component_length'] <= limit_60, genes))
    m = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , genes))
    l = list(filter(lambda t: t['component_length'] > limit_30, genes))
    
    s_precisions_bases = list(map(lambda t: t['precision_bases'] , s))
    m_precisions_bases = list(map(lambda t: t['precision_bases'] , m))
    l_precisions_bases = list(map(lambda t: t['precision_bases'] , l))
    
    s_precisions_vertex = list(map(lambda t: t['precision_vertex'] , s))
    m_precisions_vertex = list(map(lambda t: t['precision_vertex'] , m))
    l_precisions_vertex = list(map(lambda t: t['precision_vertex'] , l))
    
    
    return { 
        'small':
        {
            'e_size_density_vertex': 
            {
                'median': median(small_e_size_densities_vertex),
                'mean': mean(small_e_size_densities_vertex),
                'stdev': stdev(small_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(small_e_size_densities_bases),
                'mean': mean(small_e_size_densities_bases),
                'stdev': stdev(small_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(small_max_prop_cov_vertex),
                'mean': mean(small_max_prop_cov_vertex),
                'stdev': stdev(small_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(small_max_prop_cov_bases),
                'mean': mean(small_max_prop_cov_bases),
                'stdev': stdev(small_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(s_precisions_bases),
                'mean': mean(s_precisions_bases),
                'stdev': stdev(s_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(s_precisions_vertex),
                'mean': mean(s_precisions_vertex),
                'stdev': stdev(s_precisions_vertex)
            }
            
        }
        ,
        'medium' :
        {
            'e_size_density_vertex': 
            {
                'median': median(medium_e_size_densities_vertex),
                'mean': mean(medium_e_size_densities_vertex),
                'stdev': stdev(medium_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(medium_e_size_densities_bases),
                'mean': mean(medium_e_size_densities_bases),
                'stdev': stdev(medium_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(medium_max_prop_cov_vertex),
                'mean': mean(medium_max_prop_cov_vertex),
                'stdev': stdev(medium_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(medium_max_prop_cov_bases),
                'mean': mean(medium_max_prop_cov_bases),
                'stdev': stdev(medium_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(m_precisions_bases),
                'mean': mean(m_precisions_bases),
                'stdev': stdev(m_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(m_precisions_vertex),
                'mean': mean(m_precisions_vertex),
                'stdev': stdev(m_precisions_vertex)
            }
        },
        'large':
        {
            'e_size_density_vertex': 
            {
                'median': median(large_e_size_densities_vertex),
                'mean': mean(large_e_size_densities_vertex),
                'stdev': stdev(large_e_size_densities_vertex)
            },
            'e_size_density_bases': 
            {
                'median': median(large_e_size_densities_bases),
                'mean': mean(large_e_size_densities_bases),
                'stdev': stdev(large_e_size_densities_bases)
            },
            'max_prop_cov_vertex':
            {
                'median': median(large_max_prop_cov_vertex),
                'mean': mean(large_max_prop_cov_vertex),
                'stdev': stdev(large_max_prop_cov_vertex)
            },
            'max_prop_cov_bases':
            {
                'median': median(large_max_prop_cov_bases),
                'mean': mean(large_max_prop_cov_bases),
                'stdev': stdev(large_max_prop_cov_bases)
            },
            'precision_bases':
            {
                'median': median(l_precisions_bases),
                'mean': mean(l_precisions_bases),
                'stdev': stdev(l_precisions_bases)
            },
            'precision_vertex':
            {
                'median': median(l_precisions_vertex),
                'mean': mean(l_precisions_vertex),
                'stdev': stdev(l_precisions_vertex)
            }
        }
    }

pprint(compute_fixed_rd_table_unitigs(limit_size_60, limit_size_30, components, vertices_inv))

{'large': {'e_size_density_bases': {'mean': 0.18026949306206166,
                                    'median': 0.14033238209736415,
                                    'stdev': 0.13997914340034226},
           'e_size_density_vertex': {'mean': 0.13705535107329392,
                                     'median': 0.11036424854606672,
                                     'stdev': 0.10708766295418537},
           'max_prop_cov_bases': {'mean': 0.31658525497186246,
                                  'median': 0.2776213871114721,
                                  'stdev': 0.16438377976709076},
           'max_prop_cov_vertex': {'mean': 0.26935704116650216,
                                   'median': 0.24324324324324326,
                                   'stdev': 0.13602354166788033},
           'precision_bases': {'mean': 1.0, 'median': 1.0, 'stdev': 0.0},
           'precision_vertex': {'mean': 1.0, 'median': 1.0, 'stdev': 0.0}},
 'medium': {'e_size_density_bases': {'mean': 0.30025513001599

In [14]:
def compute_time_table(over_width, limit_60, limit_30, components, vertices_inv, algorithm_experiments='experiments'):
    time = list()
    for i, component in enumerate(components):
        if component['len'] > 2:
            width = component['width']
            l = str(width+over_width)
            if over_width == -1:
                l = str(len(component['transcript_paths']))
            if over_width == -2:
                l = str(2*component['width'])
            if component[algorithm_experiments].get(l, None) is not None:
                safe_paths_info = component[algorithm_experiments][l]

                d = dict()
                d['time'] = component[algorithm_experiments][l]['time_main']+component[algorithm_experiments][l]['time_filter']
                d['component_length'] = component['len']
                time.append(d)
    
    small = list(filter(lambda t: t['component_length'] <= limit_60, time))
    medium = list(filter(lambda t: t['component_length'] > limit_60 and t['component_length'] <= limit_30 , time))
    large = list(filter(lambda t: t['component_length'] > limit_30, time))
    
    return { 
        'small':
        {
            'time': sum(list(map(lambda x: x['time'] , small)))
        }
        ,
        'medium' :
        {
            'time': sum(list(map(lambda x: x['time'] , medium)))
        },
        'large':
        {
            'time': sum(list(map(lambda x: x['time'] , large)))
        },
        'all':
        {
            'time': sum(list(map(lambda x: x['time'] , time)))
        }
    }

pprint(compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv))
pprint(compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger'))
pprint(compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized'))
pprint(compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic'))

{'all': {'time': 14502030},
 'large': {'time': 8879561},
 'medium': {'time': 4516969},
 'small': {'time': 1105500}}
{'all': {'time': 15228484},
 'large': {'time': 9685377},
 'medium': {'time': 4552824},
 'small': {'time': 990283}}
{'all': {'time': 28716423},
 'large': {'time': 20595400},
 'medium': {'time': 7017963},
 'small': {'time': 1103060}}
{'all': {'time': 15200885},
 'large': {'time': 9034900},
 'medium': {'time': 4945210},
 'small': {'time': 1220775}}


In [15]:
## And peak memory
max(list(map(lambda x: 0 if x.get('experiments', None) is None else max(list(map(lambda k: x['experiments'][k]['peak_memory'], x['experiments'].keys()))),
             components))), max(list(map(lambda x: 0 if x.get('experiments_two_finger', None) is None else max(list(map(lambda k: x['experiments_two_finger'][k]['peak_mem'], x['experiments_two_finger'].keys()))),
             components))), max(list(map(lambda x: 0 if x.get('experiments_unoptimized', None) is None else max(list(map(lambda k: x['experiments_unoptimized'][k]['peak_mem'], x['experiments_unoptimized'].keys()))),
             components))), max(list(map(lambda x: 0 if x.get('experiments_heuristic', None) is None else max(list(map(lambda k: x['experiments_heuristic'][k]['peak_mem'], x['experiments_heuristic'].keys()))),
             components)))

(3940, 3912, 3936, 4092)

In [16]:
## LATEX TABLE GENERATORS

In [17]:
import math
def trunc(number, decimals):
    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [18]:
def merge(source, destination):
    """
    run me with nosetests --with-doctest file.py

    >>> a = { 'first' : { 'all_rows' : { 'pass' : 'dog', 'number' : '1' } } }
    >>> b = { 'first' : { 'all_rows' : { 'fail' : 'cat', 'number' : '5' } } }
    >>> merge(b, a) == { 'first' : { 'all_rows' : { 'pass' : 'dog', 'fail' : 'cat', 'number' : '5' } } }
    True
    """
    for key, value in source.items():
        if isinstance(value, dict):
            # get node or create one
            node = destination.setdefault(key, {})
            merge(value, node)
        else:
            destination[key] = value

    return destination

In [19]:
def format_number(n) :
    if type(n) == int:
        return str(n)
    return f"{n:.2f}"

def format_cell(data, key):
    
    mdata = data[key]
    return f"""\\cellx{{2}}{{{format_number(mdata['abs_improvements_base']['mean'])}\\\\{format_number(mdata['rel_improvements_base']['mean'])}}}  & \\cellx{{2}}{{{format_number(mdata['abs_improvements_vertex']['mean'])}\\\\{format_number(mdata['rel_improvements_vertex']['mean'])}}}"""

def format_row(data, key):
    
    mdata = data[key]
    return f"""${key}$  & {format_cell(mdata, 'small')} & {format_cell(mdata, 'medium')}  & {format_cell(mdata, 'large')} \\\\\\hline"""

def format_first_paper_table(data):
    a = f"""\\begin{{center}}
\\begin{{tabular}}{{|c|cc|cc|cc|}}
\\hline
 & \\multicolumn{{2}}{{c|}}{{small graphs (3-{limit_size_60} vertices)}}  & \\multicolumn{{2}}{{c|}}{{medium graphs ({limit_size_60+1}-{limit_size_30} vertices)}} & \\multicolumn{{2}}{{c|}}{{large graphs ({limit_size_30+1}-{limit_size_10} vertices)}}    \\\\
$\\ell$  & bases     & vertices      & bases     & vertices     & bases     & vertices     \\\\\\hline\\hline
{format_row(data, 'k')}
{format_row(data, 'k+1')}
{format_row(data, 't')}
{format_row(data, '2k')}
\\end{{tabular}} 
\\end{{center}}"""
    return a


d = {
    'k': {'small': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    'k+1': {'small': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    't': {'small': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    '2k': {'small': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'abs_improvements_base': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'abs_improvements_vertex': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}}
}

d2 = {
    'k': {'small': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'medium': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'large': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}}},
    'k+1': {'small': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'medium': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'large': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}}},
    't': {'small': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'medium': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'large': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}}},
    '2k': {'small': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'medium': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}},'large': {'rel_improvements_base': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}, 'rel_improvements_vertex': {'mean':  0.24, 'median': 0.22, 'stdev': 0.1}}}
}
print(format_first_paper_table(merge(d,d2)))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small graphs (3-20 vertices)}  & \multicolumn{2}{c|}{medium graphs (21-56 vertices)} & \multicolumn{2}{c|}{large graphs (57-279 vertices)}    \\
$\ell$  & bases     & vertices      & bases     & vertices     & bases     & vertices     \\\hline\hline
$k$  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} \\\hline
$k+1$  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} \\\hline
$t$  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} \\\hline
$2k$  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24}  & \cellx{2}{0.23\\0.24} \\\hline
\end{tabular} 
\end{cent

In [20]:
abs_impr_dict = dict()

abs_impr_dict['k'] = compute_abs_improvement_table(0, limit_size_60, limit_size_30, components, vertices_inv)
abs_impr_dict['k+1'] = compute_abs_improvement_table(1, limit_size_60, limit_size_30, components, vertices_inv)
abs_impr_dict['t'] = compute_abs_improvement_table(-1, limit_size_60, limit_size_30, components, vertices_inv)
abs_impr_dict['2k'] = compute_abs_improvement_table(-2, limit_size_60, limit_size_30, components,vertices_inv)

In [21]:
rel_impr_dict = dict()

rel_impr_dict['k'] = compute_rel_improvement_table(0, limit_size_60, limit_size_30, components, vertices_inv)
rel_impr_dict['k+1'] = compute_rel_improvement_table(1, limit_size_60, limit_size_30, components, vertices_inv)
rel_impr_dict['t'] = compute_rel_improvement_table(-1, limit_size_60, limit_size_30, components, vertices_inv)
rel_impr_dict['2k'] = compute_rel_improvement_table(-2, limit_size_60, limit_size_30, components,vertices_inv)

In [23]:
impr_dict = merge(abs_impr_dict, rel_impr_dict)
print(format_first_paper_table(impr_dict))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small graphs (3-20 vertices)}  & \multicolumn{2}{c|}{medium graphs (21-56 vertices)} & \multicolumn{2}{c|}{large graphs (57-279 vertices)}    \\
$\ell$  & bases     & vertices      & bases     & vertices     & bases     & vertices     \\\hline\hline
$k$  & \cellx{2}{788.81\\4.02}  & \cellx{2}{4.19\\2.31} & \cellx{2}{932.10\\6.47}  & \cellx{2}{8.09\\3.34}  & \cellx{2}{719.16\\6.96}  & \cellx{2}{9.58\\3.72} \\\hline
$k+1$  & \cellx{2}{157.47\\1.93}  & \cellx{2}{0.74\\1.24} & \cellx{2}{186.28\\2.64}  & \cellx{2}{1.49\\1.46}  & \cellx{2}{157.77\\2.85}  & \cellx{2}{2.08\\1.62} \\\hline
$t$  & \cellx{2}{157.38\\1.93}  & \cellx{2}{0.74\\1.24} & \cellx{2}{185.79\\2.64}  & \cellx{2}{1.49\\1.46}  & \cellx{2}{157.00\\2.85}  & \cellx{2}{2.07\\1.62} \\\hline
$2k$  & \cellx{2}{157.36\\1.93}  & \cellx{2}{0.74\\1.24} & \cellx{2}{185.79\\2.64}  & \cellx{2}{1.49\\1.46}  & \cellx{2}{157.00\\2.85}  & \cellx{2}{2.07\\1.62} \\\hline


In [24]:
def format_number(n) :
    if type(n) == int:
        return str(n)
    return f"{trunc(n,2):.2f}"

def format_cell(data, key, variant):
    
    mdata = data[key]
    return f"""\\cell{{2}}{{{format_number(mdata['e_size_density'+variant]['mean'])}\\\\{format_number(mdata['e_size_density'+variant]['median'])}}} & \\cell{{2}}{{{format_number(mdata['max_prop_cov'+variant]['mean'])}\\\\{format_number(mdata['max_prop_cov'+variant]['median'])}}}"""

def format_row(data, key, variant):
    
    mdata = data[key]
    return f"""{format_cell(mdata, 'small', variant)} & {format_cell(mdata, 'medium', variant)} & {format_cell(mdata, 'large', variant)} \\\\\\hline"""
    

def format_second_paper_table(data, variant=''):
    
    a = f"""\\begin{{center}}
\\begin{{tabular}}{{|c|cc|cc|cc|}}
\\hline
 & \\multicolumn{{2}}{{c|}}{{small (1-{limit_60} bases)}}    & \\multicolumn{{2}}{{c|}}{{medium ({limit_60+1}-{limit_30} bases)}} & \\multicolumn{{2}}{{c|}}{{large ({limit_30+1}-{limit_10} bases)}}    \\\\
$\\ell$  & \\texttt{{esr}}     & \\texttt{{mcr}}     & \\texttt{{esr}}     & \\texttt{{mcr}}    & \\texttt{{esr}}     & \\texttt{{mcr}}    \\\\\\hline\\hline
$k$  & {format_row(data, 'k', variant)}
$k+1$  & {format_row(data, 'k+1', variant)}
$t$  & {format_row(data, 't', variant)}
$2k$  & {format_row(data, '2k', variant)}\\hline
\\cell{{1}}{{$ST$-\\\\unitigs}}
   & {format_row(data, 'unitigs', variant)}
\\end{{tabular}} 
\\end{{center}}"""

    return a

d = {
    'k': {'small': {'e_size_density': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'medium': {'e_size_density': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}},'large': {'e_size_density': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.1, 'median': 0.2, 'stdev': 0.1}}},
    'k+1': {'small': {'e_size_density': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}, 'max_prop_cov': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}},'medium': {'e_size_density': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}, 'max_prop_cov': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}},'large': {'e_size_density': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}, 'max_prop_cov': {'mean': 0.3, 'median': 0.4, 'stdev': 0.3}}},
    't': {'small': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}},'medium': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}},'large': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}}},
    '2k': {'small': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}},'medium': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}},'large': {'e_size_density': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}, 'max_prop_cov': {'mean': 0.5, 'median': 0.6, 'stdev': 0.5}}},
    'unitigs': {'small': {'e_size_density': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}, 'max_prop_cov': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}},'medium': {'e_size_density': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}, 'max_prop_cov': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}},'large': {'e_size_density': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}, 'max_prop_cov': {'mean': 0.7, 'median': 0.8, 'stdev': 0.7}}},
}

print(format_second_paper_table(d))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small (1-1941 bases)}    & \multicolumn{2}{c|}{medium (1942-3617 bases)} & \multicolumn{2}{c|}{large (3618-22168 bases)}    \\
$\ell$  & \texttt{esr}     & \texttt{mcr}     & \texttt{esr}     & \texttt{mcr}    & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} & \cell{2}{0.10\\0.20} \\\hline
$k+1$  & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} & \cell{2}{0.30\\0.40} \\\hline
$t$  & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} \\\hline
$2k$  & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} & \cell{2}{0.50\\0.60} \\\hline\hline
\cell{1}{$ST$-\\unitigs}
   & \cell{2}{0.70\\0

In [25]:
fixed_l_dict = dict()
fixed_l_dict['unitigs'] = compute_fixed_l_table_unitigs(limit_60, limit_30, components, vertices_inv)
fixed_l_dict['k']  = compute_fixed_l_table(0, limit_60, limit_30, components, vertices_inv)
fixed_l_dict['k+1']  = compute_fixed_l_table(1, limit_60, limit_30, components, vertices_inv)
fixed_l_dict['t']  = compute_fixed_l_table(-1, limit_60, limit_30, components, vertices_inv)
fixed_l_dict['2k']  = compute_fixed_l_table(-2, limit_60, limit_30, components, vertices_inv)

In [26]:
print(format_second_paper_table(fixed_l_dict, '_bases'))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small (1-1941 bases)}    & \multicolumn{2}{c|}{medium (1942-3617 bases)} & \multicolumn{2}{c|}{large (3618-22168 bases)}    \\
$\ell$  & \texttt{esr}     & \texttt{mcr}     & \texttt{esr}     & \texttt{mcr}    & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{2}{0.48\\0.41} & \cell{2}{0.60\\0.56} & \cell{2}{0.34\\0.26} & \cell{2}{0.47\\0.42} & \cell{2}{0.31\\0.24} & \cell{2}{0.45\\0.40} \\\hline
$k+1$  & \cell{2}{0.44\\0.37} & \cell{2}{0.57\\0.53} & \cell{2}{0.29\\0.23} & \cell{2}{0.43\\0.38} & \cell{2}{0.24\\0.20} & \cell{2}{0.39\\0.35} \\\hline
$t$  & \cell{2}{0.44\\0.37} & \cell{2}{0.57\\0.53} & \cell{2}{0.29\\0.23} & \cell{2}{0.43\\0.38} & \cell{2}{0.24\\0.20} & \cell{2}{0.39\\0.35} \\\hline
$2k$  & \cell{2}{0.44\\0.37} & \cell{2}{0.57\\0.53} & \cell{2}{0.29\\0.23} & \cell{2}{0.43\\0.38} & \cell{2}{0.24\\0.20} & \cell{2}{0.39\\0.35} \\\hline\hline
\cell{1}{$ST$-\\unitigs}
   & \cell{2}{0.36\\0

In [28]:
## This is the fifth table
print(format_second_paper_table(fixed_l_dict, '_vertex'))

\begin{center}
\begin{tabular}{|c|cc|cc|cc|}
\hline
 & \multicolumn{2}{c|}{small (1-1941 bases)}    & \multicolumn{2}{c|}{medium (1942-3617 bases)} & \multicolumn{2}{c|}{large (3618-22168 bases)}    \\
$\ell$  & \texttt{esr}     & \texttt{mcr}     & \texttt{esr}     & \texttt{mcr}    & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{2}{0.42\\0.34} & \cell{2}{0.54\\0.50} & \cell{2}{0.28\\0.21} & \cell{2}{0.40\\0.34} & \cell{2}{0.23\\0.17} & \cell{2}{0.34\\0.29} \\\hline
$k+1$  & \cell{2}{0.37\\0.30} & \cell{2}{0.49\\0.43} & \cell{2}{0.22\\0.19} & \cell{2}{0.34\\0.31} & \cell{2}{0.16\\0.14} & \cell{2}{0.27\\0.25} \\\hline
$t$  & \cell{2}{0.37\\0.30} & \cell{2}{0.49\\0.43} & \cell{2}{0.22\\0.19} & \cell{2}{0.34\\0.31} & \cell{2}{0.16\\0.14} & \cell{2}{0.27\\0.25} \\\hline
$2k$  & \cell{2}{0.37\\0.30} & \cell{2}{0.49\\0.43} & \cell{2}{0.22\\0.19} & \cell{2}{0.34\\0.31} & \cell{2}{0.16\\0.14} & \cell{2}{0.27\\0.25} \\\hline\hline
\cell{1}{$ST$-\\unitigs}
   & \cell{2}{0.27\\0

In [29]:
def format_number(n) :
    if type(n) == int:
        return str(n)
    return f"{trunc(n,2):.2f}"

def format_cell(data, key, variant):
    
    mdata = data[key]
    return f"""\\cell{{1}}{{{format_number(mdata['precision'+variant]['mean'])}\\\\{format_number(mdata['precision'+variant]['median'])}}}  & \\cell{{1}}{{{format_number(mdata['e_size_density'+variant]['mean'])}\\\\{format_number(mdata['e_size_density'+variant]['median'])}}}  & \\cell{{1}}{{{format_number(mdata['max_prop_cov'+variant]['mean'])}\\\\{format_number(mdata['max_prop_cov'+variant]['median'])}}}"""
    
def format_row(data, key, variant):
    
    mdata = data[key]
    return f"""{format_cell(mdata, 'small', variant)}  & {format_cell(mdata, 'medium', variant)} & {format_cell(mdata, 'large', variant)}  \\\\\\hline"""
    

def format_third_paper_table(data, variant=''):
    
    a = f"""\\begin{{center}}
    \\begin{{tabular}}{{|c|ccc|ccc|ccc|}}
\\hline
 & \\multicolumn{{3}}{{c|}}{{small graphs (3-{limit_size_60} vertices)}}        & \\multicolumn{{3}}{{c|}}{{medium graphs ({limit_size_60+1}-{limit_size_30} vertices)}}     & \\multicolumn{{3}}{{c|}}{{large graphs ({limit_size_30+1}-{limit_size_10} vertices)}}          \\\\
$\\ell$  & \\texttt{{prec}}   & \\texttt{{esr}}     & \\texttt{{mcr}}    & \\texttt{{prec}}   & \\texttt{{esr}}     & \\texttt{{mcr}}    & \\texttt{{prec}}   & \\texttt{{esr}}     & \\texttt{{mcr}}    \\\\\\hline\\hline
$k$  & {format_row(data, 'k', variant)}
$k+1$  & {format_row(data, 'k+1', variant)}
$t$  & {format_row(data, 't', variant)}
$2k$  & {format_row(data, '2k', variant)}\\hline
\\cell{{1}}{{$ST$-\\\\unitigs}} 
  & {format_row(data, 'unitigs', variant)}
\\end{{tabular}} 
\\end{{center}}"""
    
    return a

d = {
    'k': {'small': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    'k+1': {'small': {'precision': {'mean': 0.34, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    't': {'small': {'precision': {'mean': 0.45, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    '2k': {'small': {'precision': {'mean': 0.46, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}},
    'unitigs': {'small': {'precision': {'mean': 0.56, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'medium': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}},'large': {'precision': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'e_size_density': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}, 'max_prop_cov': {'mean': 0.23, 'median': 0.22, 'stdev': 0.1}}}
}

print(format_third_paper_table(d))


\begin{center}
    \begin{tabular}{|c|ccc|ccc|ccc|}
\hline
 & \multicolumn{3}{c|}{small graphs (3-20 vertices)}        & \multicolumn{3}{c|}{medium graphs (21-56 vertices)}     & \multicolumn{3}{c|}{large graphs (57-279 vertices)}          \\
$\ell$  & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22} & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  \\\hline
$k+1$  & \cell{1}{0.34\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22} & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  \\\hline
$t$  & \cell{1}{0.45\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{1}{0.23\\0.22}  & \cell{

In [30]:
fixed_rd_dict = dict()

fixed_rd_dict['k'] = compute_fixed_rd_table(0, limit_size_60, limit_size_30, components, vertices_inv)
fixed_rd_dict['k+1'] = compute_fixed_rd_table(1, limit_size_60, limit_size_30, components, vertices_inv)
fixed_rd_dict['t'] = compute_fixed_rd_table(-1, limit_size_60, limit_size_30, components, vertices_inv)
fixed_rd_dict['2k'] = compute_fixed_rd_table(-2, limit_size_60, limit_size_30, components, vertices_inv)
fixed_rd_dict['unitigs'] = compute_fixed_rd_table_unitigs(limit_size_60, limit_size_30, components, vertices_inv)

In [32]:
print(format_third_paper_table(fixed_rd_dict, '_bases'))

\begin{center}
    \begin{tabular}{|c|ccc|ccc|ccc|}
\hline
 & \multicolumn{3}{c|}{small graphs (3-20 vertices)}        & \multicolumn{3}{c|}{medium graphs (21-56 vertices)}     & \multicolumn{3}{c|}{large graphs (57-279 vertices)}          \\
$\ell$  & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{1}{0.66\\1.00}  & \cell{1}{0.72\\0.72}  & \cell{1}{0.81\\0.87}  & \cell{1}{0.69\\0.84}  & \cell{1}{0.41\\0.35}  & \cell{1}{0.54\\0.50} & \cell{1}{0.78\\0.89}  & \cell{1}{0.25\\0.19}  & \cell{1}{0.38\\0.34}  \\\hline
$k+1$  & \cell{1}{0.99\\1.00}  & \cell{1}{0.65\\0.62}  & \cell{1}{0.77\\0.79}  & \cell{1}{0.99\\1.00}  & \cell{1}{0.36\\0.31}  & \cell{1}{0.51\\0.47} & \cell{1}{0.99\\1.00}  & \cell{1}{0.22\\0.18}  & \cell{1}{0.36\\0.32}  \\\hline
$t$  & \cell{1}{1.00\\1.00}  & \cell{1}{0.65\\0.62}  & \cell{1}{0.77\\0.79}  & \cell{1}{1.00\\1.00}  & \cell{

In [33]:
## This is the sixth table
print(format_third_paper_table(fixed_rd_dict, '_vertex'))

\begin{center}
    \begin{tabular}{|c|ccc|ccc|ccc|}
\hline
 & \multicolumn{3}{c|}{small graphs (3-20 vertices)}        & \multicolumn{3}{c|}{medium graphs (21-56 vertices)}     & \multicolumn{3}{c|}{large graphs (57-279 vertices)}          \\
$\ell$  & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    & \texttt{prec}   & \texttt{esr}     & \texttt{mcr}    \\\hline\hline
$k$  & \cell{1}{0.66\\1.00}  & \cell{1}{0.64\\0.56}  & \cell{1}{0.72\\0.70}  & \cell{1}{0.68\\0.80}  & \cell{1}{0.34\\0.28}  & \cell{1}{0.46\\0.41} & \cell{1}{0.77\\0.87}  & \cell{1}{0.21\\0.17}  & \cell{1}{0.33\\0.30}  \\\hline
$k+1$  & \cell{1}{0.99\\1.00}  & \cell{1}{0.53\\0.48}  & \cell{1}{0.65\\0.60}  & \cell{1}{0.99\\1.00}  & \cell{1}{0.28\\0.24}  & \cell{1}{0.40\\0.37} & \cell{1}{0.99\\1.00}  & \cell{1}{0.18\\0.15}  & \cell{1}{0.30\\0.27}  \\\hline
$t$  & \cell{1}{1.00\\1.00}  & \cell{1}{0.53\\0.48}  & \cell{1}{0.65\\0.60}  & \cell{1}{1.00\\1.00}  & \cell{

In [34]:
def format_time(time):
    return f'{time/1000000:.2f}'

def format_fourth_table(data, rl):
    a = f"""\\begin{{table}}
\\begin{{tabular}}{{l|l|l|l|l|}}
\\cline{{2-5}}
                       & \\multicolumn{{4}}{{c|}}{{$\\ell = {rl}$}} \\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{Gene graph sets}} & Unoptimized (secs) & Two Finger (secs) & Optimized (secs) & Heuristic (secs) \\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{small, 3-{limit_size_60} vertices}} & {format_time(data[rl]['unoptimized']['small']['time'])}  & {format_time(data[rl]['two_finger']['small']['time'])} & {format_time(data[rl]['optimized']['small']['time'])} & {format_time(data[rl]['heuristic']['small']['time'])}\\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{medium, {limit_size_60+1}-{limit_size_30} vertices}} & {format_time(data[rl]['unoptimized']['medium']['time'])} & {format_time(data[rl]['two_finger']['medium']['time'])} & {format_time(data[rl]['optimized']['medium']['time'])} & {format_time(data[rl]['heuristic']['medium']['time'])}\\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{large, {limit_size_30+1}-{limit_size_10} vertices}} & {format_time(data[rl]['unoptimized']['large']['time'])} & {format_time(data[rl]['two_finger']['large']['time'])} & {format_time(data[rl]['optimized']['large']['time'])} & {format_time(data[rl]['heuristic']['large']['time'])}\\\\ \\hline
\\multicolumn{{1}}{{|l|}}{{all, 3-{limit_size_10} vertices}}  & {format_time(data[rl]['unoptimized']['all']['time'])} & {format_time(data[rl]['two_finger']['all']['time'])} & {format_time(data[rl]['optimized']['all']['time'])} & {format_time(data[rl]['heuristic']['all']['time'])}\\\\ \\hline
\\end{{tabular}}
\\end{{table}}"""
    return a

d = {
    'k': {
            'optimized': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'heuristic': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'unoptimized': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'two_finger': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            }
    },
    'k+1': {
            'optimized': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'heuristic': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'unoptimized': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            },
            'two_finger': {
                'all': {'time': 24781434},
                 'large': {'time': 23265773},
                 'medium': {'time': 922717},
                 'small': {'time': 592944}
            }
    }
}
print(format_fourth_table(d, 'k'))

\begin{table}
\begin{tabular}{l|l|l|l|l|}
\cline{2-5}
                       & \multicolumn{4}{c|}{$\ell = k$} \\ \hline
\multicolumn{1}{|l|}{Gene graph sets} & Unoptimized (secs) & Two Finger (secs) & Optimized (secs) & Heuristic (secs) \\ \hline
\multicolumn{1}{|l|}{small, 3-20 vertices} & 0.59  & 0.59 & 0.59 & 0.59\\ \hline
\multicolumn{1}{|l|}{medium, 21-56 vertices} & 0.92 & 0.92 & 0.92 & 0.92\\ \hline
\multicolumn{1}{|l|}{large, 57-279 vertices} & 23.27 & 23.27 & 23.27 & 23.27\\ \hline
\multicolumn{1}{|l|}{all, 3-279 vertices}  & 24.78 & 24.78 & 24.78 & 24.78\\ \hline
\end{tabular}
\end{table}


In [35]:
time_dict = dict()
time_dict['k'] = dict()
time_dict['k+1'] = dict()
time_dict['t'] = dict()
time_dict['2k'] = dict()

time_dict['k']['optimized'] = compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv)
time_dict['k']['two_finger'] = compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger')
time_dict['k']['unoptimized'] = compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized')
time_dict['k']['heuristic'] = compute_time_table(0, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic')


time_dict['k+1']['optimized'] = compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv)
time_dict['k+1']['two_finger'] = compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger')
time_dict['k+1']['unoptimized'] = compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized')
time_dict['k+1']['heuristic'] = compute_time_table(1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic')


time_dict['t']['optimized'] = compute_time_table(-1, limit_size_60, limit_size_30, components, vertices_inv)
time_dict['t']['two_finger'] = compute_time_table(-1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger')
time_dict['t']['unoptimized'] = compute_time_table(-1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized')
time_dict['t']['heuristic'] = compute_time_table(-1, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic')


time_dict['2k']['optimized'] = compute_time_table(-2, limit_size_60, limit_size_30, components, vertices_inv)
time_dict['2k']['two_finger'] = compute_time_table(-2, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_two_finger')
time_dict['2k']['unoptimized'] = compute_time_table(-2, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_unoptimized')
time_dict['2k']['heuristic'] = compute_time_table(-2, limit_size_60, limit_size_30, components, vertices_inv, 'experiments_heuristic')


print(format_fourth_table(time_dict, 'k'))
print(format_fourth_table(time_dict, 'k+1'))
print(format_fourth_table(time_dict, 't'))
print(format_fourth_table(time_dict, '2k'))

\begin{table}
\begin{tabular}{l|l|l|l|l|}
\cline{2-5}
                       & \multicolumn{4}{c|}{$\ell = k$} \\ \hline
\multicolumn{1}{|l|}{Gene graph sets} & Unoptimized (secs) & Two Finger (secs) & Optimized (secs) & Heuristic (secs) \\ \hline
\multicolumn{1}{|l|}{small, 3-20 vertices} & 1.55  & 1.15 & 2.00 & 1.41\\ \hline
\multicolumn{1}{|l|}{medium, 21-56 vertices} & 10.40 & 5.17 & 7.39 & 5.62\\ \hline
\multicolumn{1}{|l|}{large, 57-279 vertices} & 25.81 & 10.51 & 11.53 & 9.75\\ \hline
\multicolumn{1}{|l|}{all, 3-279 vertices}  & 37.75 & 16.83 & 20.91 & 16.78\\ \hline
\end{tabular}
\end{table}
\begin{table}
\begin{tabular}{l|l|l|l|l|}
\cline{2-5}
                       & \multicolumn{4}{c|}{$\ell = k+1$} \\ \hline
\multicolumn{1}{|l|}{Gene graph sets} & Unoptimized (secs) & Two Finger (secs) & Optimized (secs) & Heuristic (secs) \\ \hline
\multicolumn{1}{|l|}{small, 3-20 vertices} & 1.10  & 0.99 & 1.11 & 1.22\\ \hline
\multicolumn{1}{|l|}{medium, 21-56 vertices} & 7.02 & 4.55 & 4